In [86]:
from pyspark.sql import SparkSession
import os
import sys
from config import API_KEY
from datetime import datetime, timedelta
import requests
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, DateType, DoubleType, IntegerType
import pandas as pd
from pyspark.sql import Row

In [2]:
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")
print(f"PATH: {os.environ.get('PATH')}")
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-1.8'
print(f"JAVA_HOME: {os.environ.get('JAVA_HOME')}")

os.environ["PYSPARK_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\lucas\anaconda3\python.exe"

JAVA_HOME: None
PATH: c:\Users\lucas\anaconda3;C:\Users\lucas\anaconda3;C:\Users\lucas\anaconda3\Library\mingw-w64\bin;C:\Users\lucas\anaconda3\Library\usr\bin;C:\Users\lucas\anaconda3\Library\bin;C:\Users\lucas\anaconda3\Scripts;C:\Users\lucas\anaconda3\bin;C:\Users\lucas\anaconda3\condabin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.1\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp;C:\Program Files\Rockwell Software\RSCommon;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0;C:\Windows\System32\OpenSSH;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files\Git\cmd;C:\Program Files\MongoDB\Se

In [95]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()

In [89]:
spark.stop()

In [90]:
spark = SparkSession.builder \
    .appName("weatherapp") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.python.worker.memory", "4g") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.pyspark.python", r"C:\Users\lucas\anaconda3\python.exe") \
    .config("spark.pyspark.driver.python", r"C:\Users\lucas\anaconda3\python.exe") \
    .getOrCreate()
#spark.sparkContext.addFile(r"C:\Users\lucas\anaconda3\python.exe") 

In [91]:
print(spark.conf.get("spark.pyspark.python"))
print(spark.conf.get("spark.pyspark.driver.python"))

C:\Users\lucas\anaconda3\python.exe
C:\Users\lucas\anaconda3\python.exe


In [13]:
url = "http://api.weatherapi.com/v1/history.json"
api_key = API_KEY
city = 'Philadelphia'
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)
start_date = start_date.strftime("%Y-%m-%d")
params = {
    'key': api_key,
    'q': city,
    'dt': '2024-10-22' 
}

response = requests.get(url,params=params)
data = response.json()



In [29]:
end_date.date()

datetime.date(2024, 11, 22)

In [7]:
def loop_date(start_date,end_date):
    date_list = []
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += timedelta(days=1)
    return date_list

def get_city_attr(cities,start_date,end_date):
    url = "http://api.weatherapi.com/v1/history.json"
    api_key = API_KEY
    date_list = loop_date(start_date,end_date)
    measure_metrics = ['avgtemp_f','maxwind_mph','totalprecip_in']
    spark_values_store = []
    for city in cities:
        
        for date in date_list:
            params = {
                'key': api_key,
                'q': city,
                'dt': date
            }
            response = requests.get(url,params=params)
            data = response.json()
            date_store = {'date':date, 'city':city}
            for metric in measure_metrics:
                date_store[metric] = data['forecast']['forecastday'][0]['day'][metric]
            spark_values_store.append(date_store)
    return spark_values_store


In [8]:
cities = ['Philadelphia','Atlanta','Denver']

In [9]:
end_date = datetime.now()
start_date = datetime.now() - timedelta(days=90)

In [10]:
df_list = get_city_attr(cities,start_date,end_date)
    

In [26]:
df_list

[{'date': '2024-08-25',
  'city': 'Philadelphia',
  'avgtemp_f': 76.2,
  'maxwind_mph': 6.9,
  'totalprecip_in': 0.0},
 {'date': '2024-08-26',
  'city': 'Philadelphia',
  'avgtemp_f': 77.9,
  'maxwind_mph': 7.8,
  'totalprecip_in': 0.0},
 {'date': '2024-08-27',
  'city': 'Philadelphia',
  'avgtemp_f': 80.8,
  'maxwind_mph': 7.2,
  'totalprecip_in': 0.0},
 {'date': '2024-08-28',
  'city': 'Philadelphia',
  'avgtemp_f': 84.0,
  'maxwind_mph': 10.3,
  'totalprecip_in': 0.06},
 {'date': '2024-08-29',
  'city': 'Philadelphia',
  'avgtemp_f': 71.7,
  'maxwind_mph': 12.1,
  'totalprecip_in': 0.63},
 {'date': '2024-08-30',
  'city': 'Philadelphia',
  'avgtemp_f': 72.5,
  'maxwind_mph': 10.1,
  'totalprecip_in': 0.0},
 {'date': '2024-08-31',
  'city': 'Philadelphia',
  'avgtemp_f': 76.5,
  'maxwind_mph': 8.9,
  'totalprecip_in': 0.02},
 {'date': '2024-09-01',
  'city': 'Philadelphia',
  'avgtemp_f': 75.5,
  'maxwind_mph': 9.4,
  'totalprecip_in': 0.2},
 {'date': '2024-09-02',
  'city': 'Philade

In [11]:
schema = StructType([
    StructField('date', StringType(), True),
    StructField('city', StringType(), True),
    StructField('avgtemp_f', FloatType(), True),
    StructField('maxwind_mph', FloatType(), True),
    StructField('totalprecip_in', FloatType(), True)
])

In [28]:
pdf = pd.DataFrame(df_list)

In [ ]:
def clean_record(record):
    return {
        'date' : str(record['date']),
        'city' : str(record['city']),
        'avgtemp_f' : double(record['avgtemp_f']),
        'maxwind_mph' : double(record['maxwind_mph']),
        'totalprecip_in' : float(record['totalprecip_in'])
    }

In [14]:
test_record = [df_list[0]]
print(test_record)

[{'date': '2024-08-25', 'city': 'Philadelphia', 'avgtemp_f': 76.2, 'maxwind_mph': 6.9, 'totalprecip_in': 0.0}]


In [37]:
type(df_list[0]['totalprecip_in'])

float

In [55]:
test = [clean_record(x) for x in test_record]

In [61]:
test = [{'day': 'Monday', 'rain': 'yes'}]

In [74]:
schema = StructType([
    StructField('day', StringType(), True),
    StructField('rain', StringType(), True)
])

In [77]:
rdd = spark.sparkContext.parallelize(test)
print(rdd.collect())

[{'day': 'Monday', 'rain': 'yes'}]


In [75]:
weather_df = spark.createDataFrame(rdd,schema=schema)

In [76]:
weather_df.show()

Py4JJavaError: An error occurred while calling o382.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 24) (MSI.mynetworksettings.com executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:

In [28]:
spark.sparkContext.setLogLevel("DEBUG")

In [20]:
print(sys.version)

3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]


In [31]:
sys.version

'3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]'

In [32]:
from pyspark import SparkConf
conf = SparkConf()
print(conf.get("spark.pyspark.python"))  # Python interpreter for workers
print(conf.get("spark.pyspark.driver.python"))  # Python interpreter for driver

C:\Users\lucas\anaconda3\python.exe
C:\Users\lucas\anaconda3\python.exe


In [33]:
import sys
print(sys.executable)

c:\Users\lucas\anaconda3\python.exe


In [21]:
print(os.environ.get('PYSPARK_PYTHON'))

C:\Users\lucas\anaconda3\python.exe


In [23]:
df_from_csv = spark.read.csv("diet.csv", header=True)

In [25]:
df_from_csv.show()

+-------------------+--------------+--------------------+--------+--------------+-----------+---------+---------------+---------------+---------+--------+--------+----------+-------+
|              Foods|Price/ Serving|        Serving Size|Calories|Cholesterol mg|Total_Fat g|Sodium mg|Carbohydrates g|Dietary_Fiber g|Protein g|Vit_A IU|Vit_C IU|Calcium mg|Iron mg|
+-------------------+--------------+--------------------+--------+--------------+-----------+---------+---------------+---------------+---------+--------+--------+----------+-------+
|    Frozen Broccoli|        $0.16 |           10 Oz Pkg|    73.8|             0|        0.8|     68.2|           13.6|            8.5|        8|  5867.4|   160.2|       159|    2.3|
|        Carrots,Raw|        $0.07 |    1/2 Cup Shredded|    23.7|             0|        0.1|     19.2|            5.6|            1.6|      0.6|   15471|     5.1|      14.9|    0.3|
|        Celery, Raw|        $0.04 |             1 Stalk|     6.4|             0|    

In [92]:
data = [
    {"name": "John", "age": 30},
    {"name": "Jane", "age": 25}
]

In [93]:
data_rows = [Row(**x) for x in data]
spark_df = spark.createDataFrame(data_rows)

In [83]:
# Define the schema explicitly
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

In [94]:
spark_df.show()

Py4JJavaError: An error occurred while calling o508.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (MSI.mynetworksettings.com executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)